## To create the context for working by the model

In [27]:
import pandas as pd

df = pd.read_csv('drug_dataset_expanded_updated.csv')

df.fillna(0, inplace=True)

# Creating context
context = ""
for index, row in df.iterrows():
    for column, value in row.items():
        context += f"{column}: {value}\n"
    context += "\n"

## The chatbot to retrieve information from database

In [ ]:
#!pip install intel-extension-for-tensorflow[cpu]
from transformers import pipeline
import intel_extension_for_tensorflow as tf

qa_pipeline = pipeline("question-answering", model="distilbert/distilbert-base-uncased-distilled-squad")

while True:
    question = input("Ask your question: ")
    if question.lower() == "quit":
        break

    answer = qa_pipeline(question=question, context=context)
    print(f"Answer: {answer['answer']}")

Ask your question: batch number for Omeprazole
Answer: AP389
Ask your question: quit


## To send notification mails in case of expired or expiring drugs

In [ ]:
import pandas as pd
from datetime import datetime
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Email Configuration
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
SENDER_EMAIL = "pythonprojects72@gmail.com"
SENDER_PASSWORD = "tlgf qcvt bcsi rejt"

# Function to send an email
def send_email_notification(recipient_email, drug_name, batch_number, location, expiry_date, status):
    subject = f"Drug Notification: {status}"
    body = f"""
    Dear Team,

    This is a notification regarding the drug inventory:

    Drug Name: {drug_name}
    Batch Number: {batch_number}
    Location: {location}
    Expiry Date: {expiry_date}

    Status: {status}

    Please take necessary action.

    Regards,
    Drug Inventory Management System
    """
    msg = MIMEMultipart()
    msg["From"] = SENDER_EMAIL
    msg["To"] = recipient_email
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    try:
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.sendmail(SENDER_EMAIL, recipient_email, msg.as_string())
        print(f"Email sent successfully to {recipient_email} for {drug_name}")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Loading the dataset
file_path = "drug_dataset_expanded_updated.csv"
df = pd.read_csv(file_path)

today = datetime.now().date()

# Processing each drug in the dataset
for index, row in df.iterrows():
    expiry_date = datetime.strptime(row["Expiry Date"], "%Y-%m-%d").date()
    days_to_expiry = (expiry_date - today).days

    recipient_email = "sidsharma5104@gmail.com"

    if days_to_expiry < 0:  # Drug has expired
        send_email_notification(
            recipient_email=recipient_email,
            drug_name=row["Drug Name"],
            batch_number=row["Batch Number"],
            location=row["Location"],
            expiry_date=row["Expiry Date"],
            status="Expired"
        )
    elif 0 <= days_to_expiry <= 5:  # Expiry is within 5 days
        send_email_notification(
            recipient_email=recipient_email,
            drug_name=row["Drug Name"],
            batch_number=row["Batch Number"],
            location=row["Location"],
            expiry_date=row["Expiry Date"],
            status="Expiring Soon"
        )


Email sent successfully to sidsharma5104@gmail.com for Albuterol
Email sent successfully to sidsharma5104@gmail.com for Loratadine
Email sent successfully to sidsharma5104@gmail.com for Dolo


## To redistribute the expiring drugs to nearest hospital if it doesn't have it

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import math

# Hospital locations (latitude, longitude)
hospital_locations = {
    "Sterling Hospital": (23.0292, 72.5508),
    "Zydus Hospital": (23.0600, 72.5300),
    "Apollo Hospital": (23.0455, 72.5186),
    "SAL Hospital": (23.0308, 72.5465),
    "Civil Hospital": (23.0370, 72.5913),
}

# Haversine formula to calculate distance between two coordinates
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

# Find the nearest hospital
def find_nearest_hospital(current_hospital):
    current_lat, current_lon = hospital_locations[current_hospital]
    min_distance = float("inf")
    nearest_hospital = None
    for hospital, (lat, lon) in hospital_locations.items():
        if hospital != current_hospital:
            distance = haversine(current_lat, current_lon, lat, lon)
            if distance < min_distance:
                min_distance = distance
                nearest_hospital = hospital
    return nearest_hospital

# Load the dataset
file_path = "drug_dataset_expanded_updated.csv"
df = pd.read_csv(file_path)

today = datetime.now().date()

# List to hold updated rows
updated_rows = []

# Processing drugs nearing expiry
for index, row in df.iterrows():
    expiry_date = datetime.strptime(row["Expiry Date"], "%Y-%m-%d").date()
    days_to_expiry = (expiry_date - today).days

    if days_to_expiry < 0:  # Discarding expired drugs
        continue
    elif 0 <= days_to_expiry <= 5:  # Expiry is within 5 days
        drug_name = row["Drug Name"]
        current_hospital = row["Location"]
        half_quantity = row["Quantity"] // 2

        if half_quantity > 0:
            nearest_hospital = find_nearest_hospital(current_hospital)

            # Reducing quantity in the current hospital
            row["Quantity"] -= half_quantity
            updated_rows.append(row)

            # Add half quantity to the nearest hospital
            mask = (df["Drug Name"] == drug_name) & (df["Location"] == nearest_hospital)
            if mask.any():
                nearest_hospital_index = df[mask].index[0]
                df.at[nearest_hospital_index, "Quantity"] += half_quantity
            else:
                new_row = row.copy()
                new_row["Quantity"] = half_quantity
                new_row["Location"] = nearest_hospital
                updated_rows.append(new_row)
        else:
            updated_rows.append(row)
    else:
        updated_rows.append(row)

# Creating a new DataFrame from updated rows
updated_df = pd.DataFrame(updated_rows)

# Saving the updated dataset to a new CSV file
updated_file_path = "drug_dataset_updated.csv"
updated_df.to_csv(updated_file_path, index=False)
print(f"Updated dataset saved to {updated_file_path}")


Updated dataset saved to drug_dataset_updated.csv
